In [1]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import datetime
import pickle
import time
%matplotlib inline

In [2]:
p_path="/home/takeyama/pywork/ipython/2016-05-30/"

In [3]:
cd ~/Documents/SyncRecord/cleaning-addingLABEL/

/home/takeyama/Documents/SyncRecord/cleaning-addingLABEL


クラスの問題について、レビューを頂いたのでメモする。
・オブジェクトをセンサデータ１つにしたことに対して問題がある。
センサ間のJS、KLダイバージェンスを調べるときに内部のインスタンスを呼ばないと大変
→１つのクラスですべてのセンサデータを扱える方がいい

問題に対して、クラスを位置から設計するのは大変
・クラスをまとめるクラスを用意する必要がある。

In [4]:
class AllSensorData:        
    def __init__(self):
            self._DicSensor={}
    
    def regist(self, Sname, Pdata):
        if len(self._DicSensor)==7:
            print "this class has max data"
        else:
            data = SensorData()
            data.ImportCSV(Sname,Pdata)   
            self._DicSensor[Sname]=data
            
    def ExecFFT(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        self._DicSensor[Sname].CalcFFT(samp)

    def DispFFT(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        col = self._DicSensor[Sname].GetColumns()
        for axis in col:
            print self._DicSensor[Sname].GetFFT('fft_'+axis,samp)    
            
    def ExecPower(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        self._DicSensor[Sname].CalcPower(samp)

    def DispPower(self,Sname,samp):
        print self._DicSensor[Sname].ClassName
        col = self._DicSensor[Sname].GetColumns()
        for axis in col:
            print self._DicSensor[Sname].GetPower('power_'+axis,samp)
            
    def ShowData(self,Sname):
        print self._DicSensor[Sname].ShowAllDf()
        
    # Kallback Librar Divergence
    def _KLD(self,vect1,vect2):
        f =lambda p,q : np.sum(p * np.log(p / q), axis=(p.ndim - 1))
        kld_array=np.array([])
        for vector1 in vect1:
            for vector2 in vect2:
                tmp = f( vector1, vector2 )
                kld_array = np.append(kld_array,tmp)
        return kld_array
    
    def ExecKLD(self,Sarray1,Saxis1,Sarray2,Saxis2,samp):
        start = time.time()
        dist1 = self._DicSensor[Sarray1].GetPower('power_'+Saxis1,samp)
        dist2 = self._DicSensor[Sarray1].GetPower('power_'+Saxis2,samp)
        
        kld = self._KLD(dist1,dist2)    
        np.savez(p_path+'kld/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2,kld)
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
        
    def ShowKLD(self,Sarray1,Saxis1,Sarray2,samp):
        tmp = np.load(p_path+'kld/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'.npz')['arr_0']
        print Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'_'+'Kullback–Leibler Divergence'
        print 'shape ='+str(tmp.shape)
        print tmp
        
    def _JSD(self,vect1,vect2):
        f =lambda p,q : 0.5*self._KLD(p,(0.5*(p+q)) )+0.5*self._KLD(q,(0.5*(p+q)) )
        kld_array=np.array([])
        for vector1 in vect1:
            for vector2 in vect2:
                tmp = f( vector1, vector2 )
                kld_array = np.append(kld_array,tmp)
        return kld_array
    
# Janson Shanon Divergence
    def ExecJSD(self,Sarray1,Saxis1,Sarray2,samp):
        start = time.time()
        dist1 = self._DicSensor[Sarray1].GetPower('power_'+Saxis1,samp)
        dist2 = self._DicSensor[Sarray1].GetPower('power_'+Saxis2,samp)
        
        jsd = self._JSD(dist1,dist2)    
        np.savez(p_path+'jsd/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2,kld)
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
        
    def ShowJSD(self,Sarray1,Sarray2,samp):
        tmp = np.load(p_path+'jsd/'+Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'.npz')['arr_0']
        print Sarray1+'_'+Saxis1+'_VS_'+Sarray2+'_'+Saxis2+'_'+'Jensen-Shannon Divergence'
        print 'shape ='+str(tmp.shape)
        print tmp
        

In [5]:
class SensorData:
    
    def __init__(self):
        print "__class__"
        # raw data
        self._RawData={}    
        # fft data
        self._FFTData={}
        # power spectol data
        self._PowerData={}    
        # flag exsist data
        self._Flag_exist_data=False
        
        self._columns=['AccX','AccY','AccZ','GyrX','GyrY','GyrZ']
        self._fft_col=['fft_AccX','fft_AccY','fft_AccZ','fft_GyrX','fft_GyrY','fft_GyrZ']
        self._power_col=['power_AccX','power_AccY','power_AccZ','power_GyrX','power_GyrY','power_GyrZ']
        
    def ImportCSV(self,Sclass,csv_file):
        self.ClassName=Sclass
        self.Flag_exist_data=True
        # design dataframe
        data = pd.read_csv(csv_file,encoding="SHIFT-JIS")
        data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
        data.Time=pd.to_datetime(data.Time)
        data = pd.pivot_table(data,values=[u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ'],index=[u'Time'])
        
        # convert numpy.darray
        AccX=data.AccX.values*0.0001
        AccY=data.AccY.values*0.0001
        AccZ=data.AccZ.values*0.0001
        GyrX=data.GyrX.values*0.01
        GyrY=data.GyrY.values*0.01
        GyrZ=data.GyrZ.values*0.01
        Time=data.index.to_pydatetime().astype('datetime64[ns]')
        
        # regist each raw data 
        self._RawData['AccX'] = AccX
        self._RawData['AccY'] = AccY
        self._RawData['AccZ'] = AccZ
        self._RawData['GyrX'] = GyrX
        self._RawData['GyrY'] = GyrY
        self._RawData['GyrZ'] = GyrZ
        self._RawData['Time'] = Time
    
    def ShowFlagExistData(self):
        return self.Flag_exist_data
    
    def GetColumns(self):
        return self._columns

    def ShowAllDf(self):
        print 'AccX : ';print self._RawData['AccX']
        print 'AccY : ';print self._RawData['AccY']
        print 'AccZ : ';print self._RawData['AccZ']
        print 'GyrX : ';print self._RawData['GyrX']
        print 'GyrY : ';print self._RawData['GyrY']
        print 'GyrZ : ';print self._RawData['GyrZ']
        
    def _Time2Num(self,time):
        return np.where(self._RawData['Time']==np.datetime64(time) )[0][0]
    
    def ShowQuery(self,Sname,rng=[]):
        data = self._RawData[Sname]
        print Sname+':'+str( data[rng[0]:rng[1]])
            
    def _sliding_window(self,Sname,samp, overlap):
        count =0
        s =self._RawData['Time'][0]
        start=self._Time2Num(s)
        g = s+np.timedelta64(samp*10,'ms')
        goal= self._Time2Num(g)
        yield self._RawData[Sname][start:goal]
        
        add=overlap*0.01
        
        while True:
            try:
                count +=1
                s =s+np.timedelta64(samp*10,'ms')
                start=self._Time2Num(s)
                g  =s+np.timedelta64(samp*10,'ms')
                goal= self._Time2Num(g)
                yield self._RawData[Sname][start:goal]
            except StopIteration:
                print '_sliding_window StopIteration'
                break   
            except IndexError:
                print '_sliding_window IndexError'
                break  
                
# Fast Frier transaction            
    def GetFFT(self,Sfft,samp):
        return np.load(p_path+'fft/'+self.ClassName+'_'+Sfft+'_'+str(samp)+'.npz')['arr_0']
    
    def CalcFFT(self,samp,overlap=0.5):
        start = time.time()
        fft_data = np.array([])
        
        for n,f in zip( self._columns,self._fft_col):
            print 'start'+n+'->'+f
            sw = self._sliding_window(n,samp,overlap)
            while True:
                try:
                    d=sw.next()
                    fft_data = np.append(fft_data, fftpack.fft(d)[1:(samp/2)+1] )   # fftの直流成分を除くsample/2の
                except StopIteration:
                    print 'CalcFFTStopIteration'
                    fft_data = fft_data.reshape(len(fft_data)/(samp/2),(samp/2) )
                    self._FFTData[f] = fft_data
                    np.savez(p_path+'fft/'+self.ClassName+'_'+str(f)+'_'+str(samp),self._FFTData[f])
                    break   
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
           

# Spectol Power 
    def GetPower(self,Spower,samp):
        return np.load(p_path+'power/'+self.ClassName+'_'+Spower+'_'+str(samp)+'.npz')['arr_0']
    
    def _power(self,fft_array):
        p=lambda x,y : np.sqrt(x**2+y**2)
        power_array=np.array([])
        for vector in fft_array:
            tmp = p( np.real(vector),np.imag(vector) )
            tmp = tmp/np.sum(tmp)
            power_array = np.append(power_array,tmp)
        return power_array
        
    def CalcPower(self,samp,overlap=0.5):
        start = time.time()
        for fft_name,power_name in zip( self._fft_col, self._power_col):
            print 'start'+fft_name+'->'+power_name
            fft_data = self.GetFFT(fft_name,samp)
            power_data=self._power(fft_data)
            np.savez(p_path+'power/'+self.ClassName+'_'+power_name+'_'+str(samp),power_data)
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"

In [7]:
AllTest = AllSensorData()
AllTest.regist('left_hand','Conv-left-hand-ags.csv')
AllTest.regist('right_leg','Conv-right-leg-ags.csv')
AllTest.regist('left_leg','Conv-left-leg-ags.csv')
AllTest.regist('right_hand','Conv-right-hand-ags.csv')

__class__
__class__
__class__
__class__


In [11]:
AllTest.ShowData('left_hand')

AccX : 
[ 0.8226  0.8058  0.8292 ..., -0.1397 -0.1346 -0.1292]
AccY : 
[ 0.7922  0.8437  0.9091 ...,  0.7067  0.7526  0.8188]
AccZ : 
[-0.0543 -0.0582 -0.0231 ..., -0.5123 -0.4901 -0.5248]
GyrX : 
[ 353.11  361.41  353.95 ...,   -4.59   -2.53   -2.67]
GyrY : 
[ 71.96  59.74  48.03 ...,  56.    57.86  60.15]
GyrZ : 
[ -9.53 -22.7  -34.02 ..., -36.9  -32.73 -29.73]
None


In [12]:
AllTest.ShowData('left_leg')

AccX : 
[ 0.9501  0.9526  0.9584 ...,  0.9294  0.967   0.8093]
AccY : 
[ 0.1694  0.1709  0.1829 ..., -0.454  -0.5392 -0.1962]
AccZ : 
[-0.0438 -0.0521 -0.0445 ...,  0.5364  0.5989 -0.1353]
GyrX : 
[  6.08   7.53   9.67 ...,  62.94  71.18  83.5 ]
GyrY : 
[-15.73 -16.93 -18.4  ...,  37.97  54.34  46.68]
GyrZ : 
[-5.98 -6.38 -6.61 ..., -8.83  0.36 -5.52]
None


In [13]:
AllTest.ShowData('right_leg')

AccX : 
[ 0.8565  0.8391  0.8926 ...,  0.9554  0.9534  0.9619]
AccY : 
[ 0.0087 -0.0596 -0.0813 ..., -0.28   -0.2688 -0.2673]
AccZ : 
[ 0.209   0.157   0.1709 ...,  0.0203  0.0527  0.0481]
GyrX : 
[-21.33 -23.51 -14.14 ...,   7.81   7.21   6.54]
GyrY : 
[ -4.79   7.47  17.92 ...,   6.63   5.67   5.02]
GyrZ : 
[ 13.29   5.15  -5.34 ...,  -7.26  -7.03  -6.23]
None


In [14]:
AllTest.ShowData('right_hand')

AccX : 
[ 0.8258  0.8805  0.9398 ...,  0.0316  0.0463  0.0057]
AccY : 
[-0.3066 -0.2997 -0.2975 ..., -0.0693 -0.1137 -0.0734]
AccZ : 
[ 0.1603  0.1664  0.1911 ...,  1.1037  1.0844  1.0234]
GyrX : 
[-158.22 -152.1  -146.42 ...,  -13.93  -15.14  -11.08]
GyrY : 
[-18.43 -23.89 -28.76 ..., -13.11 -12.04  -9.8 ]
GyrZ : 
[-87.73 -89.44 -92.54 ...,  42.57  42.61  41.56]
None


In [16]:
AllTest.ExecFFT('left_hand',16)
AllTest.ExecFFT('left_leg',16)

left_hand
startAccX->fft_AccX
_sliding_window IndexError
CalcFFTStopIteration


IOError: [Errno 13] Permission denied: '/home/takeyama/pywork/ipython/2016-05-30/fft/left_hand_fft_AccX_16.npz'

In [17]:
AllTest.ExecFFT('right_leg',16)
AllTest.ExecFFT('right_hand',16)

right_leg
startAccX->fft_AccX
_sliding_window IndexError
CalcFFTStopIteration


IOError: [Errno 13] Permission denied: '/home/takeyama/pywork/ipython/2016-05-30/fft/right_leg_fft_AccX_16.npz'

In [ ]:
AllTest.DispFFT('left_hand',16)

In [ ]:
AllTest.DispFFT('right_leg',16)

In [ ]:
AllTest.ExecPower('left_hand',16)
AllTest.ExecPower('right_leg',16)

In [ ]:
AllTest.ExecKLD('left_hand','AccX','right_leg','AccX',samp=16)